In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text # Not used directly but needed to import TF ops.

from tensorflow.keras.layers import Input, Dot, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [2]:
path_to_model = "models/USE_mul3"

In [3]:
df = pd.read_parquet('data/not_load/qa_short.parquet'); df.head()

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
1,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Зачем эта простыня текста. Ее традиционно никт...,"[{'name': 'tnx', 'count': 1}, {'name': 'heavy_..."
2,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",> в проект Data Science\nвыглядит как bullshit...,"[{'name': '+1', 'count': 1}]"
3,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",<@U64GCA997> сильно лучше :notbad:,"[{'name': 'tnx', 'count': 1}]"
4,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Что такое бустануть? Что за нужно уметь в них?...,"[{'name': 'heavy_plus_sign', 'count': 1}, {'na..."


In [4]:
graph = pd.read_parquet('data/not_load/qa_sim.parquet').loc[:, ['question', 'q_id', 'ans_ids']]; graph.head()

,question,q_id,ans_ids
0,"Народ, кто знает, есть ли в GO библиотека для ...",0,0
1,"Всем привет, нуждаюсь в помощи по составлению ...",1,"1,2,3,4"
2,"и еще вопрос, нет ли веяний, что этот проект п...",5,"5,6"
3,Кто-нибудь здесь проходил\n<http://coursera.or...,7,7
4,"Ahh folks, anyone is able to submit an lstm+en...",8,8


Отбираем пары для отладки

In [5]:
test_pairs = df.loc[213:217, ['question', 'answer']]; test_pairs

,question,answer
213,"Привет всем опять!\n\nУ меня есть конфиг, в ко...","Привет :slightly_smiling_face: Да, все верно. ..."
214,А есть у кого нибудь внятное объяснение с како...,хм. просто мне наврерное не нравится идея все ...
215,А есть у кого нибудь внятное объяснение с како...,"но, как уже заметили можно начать и без “пайпл..."
216,А есть у кого нибудь внятное объяснение с како...,"к слову, если использовать кукикаттер и шаги +..."
217,Есть более подробный туториал как удалять данн...,"Только не пользуйтесь `-c`, если у вас один ре..."


In [6]:
test_graph = graph.loc[graph['question'].isin(test_pairs['question']), ['q_id', 'ans_ids']]; test_graph

,q_id,ans_ids
137,213,213
138,214,"214,215,216"
139,217,"217,218"


In [7]:
similarity = []
q_ids = []
for _, row in test_graph.iterrows():
    el = eval(row['ans_ids'])
    if isinstance(el, int):
        similarity.append([el])
        q_ids.append(row['q_id'])
    else:
        for i, answer in enumerate(el):
            similarity.append(list(el))
            q_ids.append(el[i])

similarity

[[213],
 [214, 215, 216],
 [214, 215, 216],
 [214, 215, 216],
 [217, 218],
 [217, 218]]

In [8]:
q_ids

[213, 214, 215, 216, 217, 218]

In [9]:
def pad_sim_list(sim, true_ids):
    """
    Pads a vector ``vec`` up to length ``length`` along axis ``dim`` with pad symbol ``pad_symbol``.
    """
    max_len = max([len(x) for x in similarity])
    mat = np.full((len(sim), max_len), -1)
    for row, column in enumerate(sim):
        mat[row, range(len(column))] = column

    return np.hstack((mat, np.expand_dims(np.array(true_ids), axis=1)))

In [10]:
similarity = pad_sim_list(similarity, q_ids); similarity

array([[213,  -1,  -1, 213],
       [214, 215, 216, 214],
       [214, 215, 216, 215],
       [214, 215, 216, 216],
       [217, 218,  -1, 217],
       [217, 218,  -1, 218]])

In [11]:
shape = len(q_ids)
labels = np.zeros((shape, shape), dtype=bool)

In [12]:
true_ids = similarity[:, -1]
similarity = np.delete(similarity, -1, 1)
for i, row in enumerate(similarity):
    for el in row:
        j = np.argwhere(true_ids == el)
        if j != []:
            j = np.argwhere(true_ids == el).item()
        labels[i, j] = True

In [13]:
labels

array([[ True, False, False, False, False, False],
       [False,  True,  True,  True, False, False],
       [False,  True,  True,  True, False, False],
       [False,  True,  True,  True, False, False],
       [False, False, False, False,  True,  True],
       [False, False, False, False,  True,  True]])

Функции потерь

In [5]:
class TripletLoss(tf.keras.losses.Loss):
    """Computes the triplet loss with self-labeled positive pairs.
    The loss encourages the positive distances (between a pair of embeddings
    with the same labels) to be smaller than the minimum negative distance
    among which are at least greater than the positive distance plus the
    margin constant (called semi-hard negative) in the mini-batch.
    If no such negative exists, uses the largest negative distance instead.
    See: https://arxiv.org/abs/1503.03832.
    We expect labels `y_true` to be provided as 1-D integer `Tensor` with shape
    [batch_size] of multi-class integer labels. And embeddings `y_pred` must be
    2-D float `Tensor` of l2 normalized embedding vectors.
    Args:
      margin: Float, margin term in the loss definition. Default value is 1.0.
      name: Optional name for the op.
    """

    def __init__(
        self, margin = 1.0, name = None, **kwargs
    ):
        super().__init__(name=name, reduction=tf.keras.losses.Reduction.NONE)
        self.margin = margin

    def call(self, similarity, dot_product):
        shape = dot_product.shape[0]
        labels = np.zeros((shape, shape), dtype=bool)
        true_ids = similarity[:, -1]
        similarity = np.delete(similarity, -1, 1)
        for i, row in enumerate(similarity):
            for el in row:
                j = np.argwhere(true_ids == el)
                if j != []:
                    j = np.argwhere(true_ids == el).item()
                labels[i, j] = True

        s_pos = dot_product*tf.cast(labels, tf.float32)
        s_neg = dot_product*tf.cast(tf.math.logical_not(tf.cast(labels, tf.bool)), tf.float32)

        loss = tf.maximum(0.0, self.margin - s_pos + s_neg)
        loss = tf.reduce_mean(loss)
        return loss

    def get_config(self):
        config = {
            "margin": self.margin,
        }
        base_config = super().get_config()
        return {**base_config, **config}

In [6]:
class CrossEntropyLoss(tf.keras.losses.Loss):
    """Computes the crossentopy loss with self-labeled positive pairs.
    Args:
      name: Optional name for the op.
    """

    def __init__(
        self, name = None, **kwargs
    ):
        super().__init__(name=name, reduction=tf.keras.losses.Reduction.NONE)
        self.cross = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    def call(self, similarity, dot_product):
        shape = dot_product.shape[0]
        labels = np.zeros((shape, shape), dtype=bool)
        true_ids = similarity[:, -1]
        similarity = np.delete(similarity, -1, 1)
        for i, row in enumerate(similarity):
            for el in row:
                j = np.argwhere(true_ids == el)
                if j != []:
                    j = np.argwhere(true_ids == el).item()
                labels[i, j] = True
        labels = tf.cast(labels, tf.float32)

        loss = self.cross(labels, dot_product)
        return loss

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')

Строим модель, получающую эмбеддинги вопросов и ответов и строящую матрицу

In [8]:
question = Input(shape=(1,), dtype=tf.string, name='Question')
answer = Input(shape=(1,), dtype=tf.string, name='Answer')
use = hub.KerasLayer(path_to_model, trainable=True, name='USE')
#encode questions and answers
q_emb = use(tf.squeeze(tf.cast(question, tf.string)))
a_emb = use(tf.squeeze(tf.cast(answer, tf.string)))
#apply cosine similarity function
dist_matrix = 1 - tf.matmul(tf.math.l2_normalize(q_emb, axis=1), tf.math.l2_normalize(tf.transpose(a_emb), axis=0))
model = Model(inputs=[question, answer], outputs=dist_matrix)
model.compile(loss=TripletLoss(), optimizer=opt, run_eagerly=True)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Question (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
Answer (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Squeeze (TensorFlow [None]               0           Question[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_Squeeze_1 (TensorFl [None]               0           Answer[0][0]                     
______________________________________________________________________________________________

In [12]:
model([test_pairs['question'].to_numpy(), test_pairs['answer'].to_numpy()])

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0.9058144 , 0.8884254 , 0.7597515 , 0.9857815 , 0.7766231 ],
       [1.0161836 , 0.82771003, 0.58489823, 0.9291676 , 0.7595733 ],
       [1.0161836 , 0.82771003, 0.58489823, 0.9291676 , 0.7595733 ],
       [1.0161836 , 0.82771003, 0.58489823, 0.9291676 , 0.7595733 ],
       [1.0161837 , 0.82771003, 0.5848982 , 0.9291676 , 0.7595732 ]],
      dtype=float32)>

### Тест обучения

In [7]:
similarity = []
true_ids = []
for _, row in graph.iterrows():
    el = eval(row['ans_ids'])
    if isinstance(el, int):
        similarity.append([el])
        true_ids.append(row['q_id'])
    else:
        for i, answer in enumerate(el):
            similarity.append(list(el))
            true_ids.append(el[i])

print(len(similarity) == len(df))

True


In [8]:
def pad_sim_list(sim, true_ids):
    """
    Pads a vector ``vec`` up to length ``length`` along axis ``dim`` with pad symbol ``pad_symbol``.
    """
    max_len = max([len(x) for x in similarity])
    mat = np.full((len(sim), max_len), -1)
    for row, column in enumerate(sim):
        mat[row, range(len(column))] = column

    return np.hstack((mat, np.expand_dims(np.array(true_ids), axis=1)))

In [15]:
train_hist = model.fit([df['question'].to_numpy(), df['answer'].to_numpy()], 
                       pad_sim_list(similarity, true_ids),
                       batch_size=64,
                       epochs=4)

Train on 49394 samples
Epoch 1/4
49394/49394 [==============================] - 2336s 47ms/sample - loss: 1.1131
Epoch 2/4
49394/49394 [==============================] - 1669s 34ms/sample - loss: 1.0004
Epoch 3/4
49394/49394 [==============================] - 1661s 34ms/sample - loss: 1.0001
Epoch 4/4
49394/49394 [==============================] - 1660s 34ms/sample - loss: 1.0000


Смотрим метрику после пробного обучения

In [22]:
from modules.evaluation import evaluate

use = model.layers[4]
questions = df['question'].unique()
answers = df['answer']
similarity = []
for st in graph['ans_ids'].to_numpy():
    el = eval(st)
    if isinstance(el, int):
        similarity.append([el])
    else:
        similarity.append(list(el))

map10 = evaluate(use, questions, answers, similarity)
print('')
print(map10)

Searching for top k texts for all inputs: 100%|██████████| 21018/21018 [13:22<00:00, 26.20it/s]
0.0017150234291494165



Пробуем с другим лоссом

In [11]:
from modules.training import train

model, train_hist = train(path_to_model, CrossEntropyLoss(), opt, 
                          [df['question'].to_numpy(), df['answer'].to_numpy()],
                          pad_sim_list(similarity, true_ids),
                          batch_size=64,
                          epochs=4)

Train on 49394 samples
Epoch 1/4
49394/49394 [==============================] - 1724s 35ms/sample - loss: 4.2306
Epoch 2/4
49394/49394 [==============================] - 1694s 34ms/sample - loss: 4.1869
Epoch 3/4
49394/49394 [==============================] - 1700s 34ms/sample - loss: 4.1953
Epoch 4/4
49394/49394 [==============================] - 2025s 41ms/sample - loss: 4.1880


Смотрим метрику

In [12]:
from modules.evaluation import evaluate

use = model.layers[4]
questions = df['question'].unique()
answers = df['answer']
similarity = []
for st in graph['ans_ids'].to_numpy():
    el = eval(st)
    if isinstance(el, int):
        similarity.append([el])
    else:
        similarity.append(list(el))

map10 = evaluate(use, questions, answers, similarity)
print('')
print(map10)

Searching for top k texts for all inputs: 100%|██████████| 21018/21018 [12:24<00:00, 28.24it/s]
0.00013569141122508077



### Пробуем обучать SBERT

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from modules.training import SBERT, SentencesDataset

In [2]:
df = pd.read_parquet('data/not_load/qa_short.parquet')
graph = pd.read_parquet('data/not_load/qa_sim.parquet').loc[:, ['question', 'q_id', 'ans_ids']]
df['question'] = df['question'].str.lower()
df['answer'] = df['answer'].str.lower()

In [3]:
model = SBERT('distiluse-base-multilingual-cased')

Using device cuda


In [4]:
train_set = df.loc[:, ['question', 'answer']].to_numpy()

In [5]:
similarity = []
true_ids = []
for _, row in graph.iterrows():
    el = eval(row['ans_ids'])
    if isinstance(el, int):
        similarity.append([el])
        true_ids.append(row['q_id'])
    else:
        for i, answer in enumerate(el):
            similarity.append(list(el))
            true_ids.append(el[i])

print(len(similarity) == len(df))

True


In [6]:
def pad_sim_list(sim, true_ids):
    """
    Pads a vector ``vec`` up to length ``length`` along axis ``dim`` with pad symbol ``pad_symbol``.
    """
    max_len = max([len(x) for x in similarity])
    mat = np.full((len(sim), max_len), -1)
    for row, column in enumerate(sim):
        mat[row, range(len(column))] = column

    return np.hstack((mat, np.expand_dims(np.array(true_ids), axis=1)))

In [7]:
ds = SentencesDataset(train_set, pad_sim_list(similarity, true_ids), model)

Convert dataset: 100%|██████████| 49394/49394 [01:16<00:00, 642.04it/s]



Sentences 0 longer than max_sequence_length: 13931
Sentences 1 longer than max_sequence_length: 4296


In [8]:
tdl = DataLoader(ds, shuffle=True, batch_size=16)

In [9]:
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, seq_a, seq_b, label):
        loss_fct = torch.nn.CosineEmbeddingLoss()
        device = seq_a.device
        y = torch.tensor([1]*seq_a.size(0)).to(device)

        loss = loss_fct(seq_a, seq_b, y)
        return loss

In [10]:
class MultipleNegativesRankingLoss(nn.Module):
    def __init__(self, margin: float = 1.0):
        super(MultipleNegativesRankingLoss, self).__init__()
        self.margin = margin

    def forward(self, seq_a, seq_b, similarity):
        scores = torch.matmul(seq_a, seq_b.t())
        
        shape = scores.shape[0]
        labels = np.zeros((shape, shape), dtype=bool)
        similarity = similarity.detach().cpu().numpy()
        true_ids = similarity[:, -1]
        similarity = np.delete(similarity, -1, 1)
        for i, row in enumerate(similarity):
            for el in row:
                j = np.argwhere(true_ids == el)
                if j != []:
                    j = np.argwhere(true_ids == el).item()
                labels[i, j] = True
        labels = torch.tensor(labels).to(scores.device)
        
        s_pos = scores*labels
        s_neg = scores*(~labels)
        
        loss = torch.clamp(self.margin - s_pos + s_neg, min=0.0)
        loss = torch.mean(loss)
        return loss

In [9]:
train_objective = (tdl, CosineSimilarityLoss())
model.fit(train_objective, epoches=4)

Epoch 4 Loss 6.27e-06: 100%|██████████| 3088/3088 [21:40<00:00,  2.38it/s]


Смотрим результат

In [4]:
from modules.evaluation import evaluate

questions = df['question'].unique()
answers = df['answer'].to_numpy()
similarity = []
for st in graph['ans_ids'].to_numpy():
    el = eval(st)
    if isinstance(el, int):
        similarity.append([el])
    else:
        similarity.append(list(el))

In [5]:
map10 = evaluate(model, questions, answers, similarity, model_type='sbert', batch_size=64)
print('')
print(map10)

Searching for top k texts for all inputs: 100%|██████████| 21018/21018 [14:49<00:00, 23.63it/s]
0.0005444130528581951



Пробуем триплет лосс

In [11]:
train_objective = (tdl, MultipleNegativesRankingLoss())
model.fit(train_objective, epoches=4)

Epoch 4 Loss 0.162: 100%|██████████| 3088/3088 [22:40<00:00,  2.27it/s]


In [4]:
from modules.evaluation import evaluate

questions = df['question'].unique()
answers = df['answer'].to_numpy()
similarity = []
for st in graph['ans_ids'].to_numpy():
    el = eval(st)
    if isinstance(el, int):
        similarity.append([el])
    else:
        similarity.append(list(el))
        
map10 = evaluate(model, questions, answers, similarity, model_type='sbert', batch_size=64)
print('')
print(map10)

Searching for top k texts for all inputs: 100%|██████████| 21018/21018 [12:12<00:00, 28.69it/s]
8.815195440944797e-05



Заготовка тренировочного велосипеда

In [18]:
# Prepare the training dataset.
batch_size = 2
q = test_pairs['question'].to_numpy()
a = test_pairs['answer'].to_numpy()
train_dataset = tf.data.Dataset.from_tensor_slices((q, a, pad_sim_list(similarity)))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [19]:
trl = TripletLoss()
for batch in train_dataset:
    with tf.GradientTape() as tape:
      logits = model((batch[0], batch[1]))
      loss_value = trl(batch[2], logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

tf.Tensor(
[[0.6475699  0.2517897 ]
 [0.6475699  0.25178975]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[1.258095   0.        ]
 [0.34368467 0.6485029 ]], shape=(2, 2), dtype=float32)
tf.Tensor([[1.1938863]], shape=(1, 1), dtype=float32)


### Классификация

In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text # Not used directly but needed to import TF ops.

from tensorflow.keras.layers import Input, Dot, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [2]:
path_to_model = "models/USE_mul3"
df = pd.read_parquet('data/qa_classif.parquet'); df.head()

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","В python. Аналог pandas есть только в R, но та...",nan
1,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
2,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...",В Julia.,nan
3,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...",Вообще есть такая поделка - <https://github.co...,nan
4,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Ну и arrow есть, правда пока там мало что можн...",nan


In [3]:
indxs = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    if any(['proj' in row['channel'], 'meetings' in row['channel']]):
        indxs.append(i)

print(f'\nОтветов выбросить {len(df.loc[indxs])}, количество записей станет {len(df)-len(df.loc[indxs])}')

100%|██████████| 398292/398292 [00:32<00:00, 12403.68it/s]

Ответов выбросить 38244, количество записей станет 360048


In [4]:
df.drop(index=indxs, axis=0, inplace=True)
df.drop(index=[df[df['channel'] == 'ods_challenges'].index[0],
               df[df['channel'] == '_top_jobs'].index[0],
               df[df['channel'] == 'welcome'].index[0]], axis=0, inplace=True)

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['channel'].value_counts()[df['channel'].value_counts() > 100])

career                    65991
_random_talks             40323
deep_learning             32524
theory_and_practice       27765
mlcourse_ai_news          20659
lang_python               20642
nlp                       16848
hardware                  14236
kaggle_crackers            8038
edu_courses                6640
ods_sport                  6377
mlcourse_ai                5685
mltrainings_beginners      5658
devops                     5622
big_data                   5385
cv                         5355
_random_b                  5333
datasets                   4058
blockchain                 3658
gnomiki                    3373
satellite_imaging          3145
class_dl_cmu               2908
lang_r                     2728
visualization              2494
trading                    1990
class_cs231n               1961
edu_books                  1938
ml_pipeline                1817
tool_catalyst              1680
_jobs                      1624
ods_habr                   1574
mlcourse

In [11]:
to_drop = df['channel'].value_counts()[df['channel'].value_counts() <= 100].index
df.drop(index=df[df['channel'].isin(to_drop)].index, axis=0, inplace=True)

In [12]:
df_labels = pd.get_dummies(df['channel'])

In [13]:
class_weights = {}
total = df['channel'].nunique()
for i, num in enumerate(df['channel'].value_counts().sort_index()):
    class_weights.update({i: total/num})

In [14]:
import pickle

with open('data/label_weights.pkl', 'wb') as f:
    pickle.dump(class_weights, f)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df['question'], df_labels, test_size=0.2, shuffle=True, stratify=df_labels)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True, stratify=y_train)

In [17]:
train = pd.concat([X_train, y_train], axis=1, ignore_index=True); train.columns = ['question', *df_labels.columns]

In [18]:
val = pd.concat([X_val, y_val], axis=1, ignore_index=True); val.columns = ['question', *df_labels.columns]

In [19]:
test = pd.concat([X_test, y_test], axis=1, ignore_index=True); test.columns = ['question', *df_labels.columns]

In [20]:
train.to_parquet('data/train.parquet', compression='brotli', index=False)
val.to_parquet('data/val.parquet', compression='brotli', index=False)
test.to_parquet('data/test.parquet', compression='brotli', index=False)

In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')

In [11]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0,  
                                                  name='categorical_crossentropy')

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', verbose=1,
                                                  patience=4, mode='max', restore_best_weights=True)
checkpoint = ModelCheckpoint('models/USE_classif.h5', monitor='val_auc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')
reduse_lr = ReduceLROnPlateau(monitor='val_auc', factor=0.1, patience=2, verbose=1, mode='max',
                              min_delta=0.001, cooldown=1, min_lr=1e-9)

In [13]:
from tensorflow.python.distribute import distribution_strategy_context as distribute_ctx
from tensorflow.python.eager import context, def_function
from tensorflow.python.framework import constant_op, tensor_spec, tensor_shape, ops, dtypes
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.keras.utils.generic_utils import deserialize_keras_object
from tensorflow.python.keras.utils.generic_utils import serialize_keras_object
from tensorflow.python.keras.utils.generic_utils import to_list
from tensorflow.python.keras.utils.tf_utils import is_tensor_or_variable
from tensorflow.python.ops import array_ops, weights_broadcast_ops, confusion_matrix, check_ops, control_flow_ops, math_ops, init_ops, nn
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.ops.losses import util as tf_losses_utils
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.util import nest, tf_inspect
from tensorflow.python.util.tf_export import keras_export
from tensorflow.tools.docs import doc_controls

class AUC(tf.keras.metrics.Metric):
  """Computes the approximate AUC (Area under the curve) via a Riemann sum.
  This metric creates four local variables, `true_positives`, `true_negatives`,
  `false_positives` and `false_negatives` that are used to compute the AUC.
  To discretize the AUC curve, a linearly spaced set of thresholds is used to
  compute pairs of recall and precision values. The area under the ROC-curve is
  therefore computed using the height of the recall values by the false positive
  rate, while the area under the PR-curve is the computed using the height of
  the precision values by the recall.
  This value is ultimately returned as `auc`, an idempotent operation that
  computes the area under a discretized curve of precision versus recall values
  (computed using the aforementioned variables). The `num_thresholds` variable
  controls the degree of discretization with larger numbers of thresholds more
  closely approximating the true AUC. The quality of the approximation may vary
  dramatically depending on `num_thresholds`. The `thresholds` parameter can be
  used to manually specify thresholds which split the predictions more evenly.
  For best results, `predictions` should be distributed approximately uniformly
  in the range [0, 1] and not peaked around 0 or 1. The quality of the AUC
  approximation may be poor if this is not the case. Setting `summation_method`
  to 'minoring' or 'majoring' can help quantify the error in the approximation
  by providing lower or upper bound estimate of the AUC.
  If `sample_weight` is `None`, weights default to 1.
  Use `sample_weight` of 0 to mask values.
  Usage:
  >>> m = tf.keras.metrics.AUC(num_thresholds=3)
  >>> _ = m.update_state([0, 0, 1, 1], [0, 0.5, 0.3, 0.9])
  >>> # threshold values are [0 - 1e-7, 0.5, 1 + 1e-7]
  >>> # tp = [2, 1, 0], fp = [2, 0, 0], fn = [0, 1, 2], tn = [0, 2, 2]
  >>> # recall = [1, 0.5, 0], fp_rate = [1, 0, 0]
  >>> # auc = ((((1+0.5)/2)*(1-0))+ (((0.5+0)/2)*(0-0))) = 0.75
  >>> m.result().numpy()
  0.75
  >>> m.reset_states()
  >>> _ = m.update_state([0, 0, 1, 1], [0, 0.5, 0.3, 0.9],
  ...                    sample_weight=[1, 0, 0, 1])
  >>> m.result().numpy()
  1.0
  Usage with tf.keras API:
  ```python
  model = tf.keras.Model(inputs, outputs)
  model.compile('sgd', loss='mse', metrics=[tf.keras.metrics.AUC()])
  ```
  """

  def __init__(self,
               num_thresholds=200,
               curve='ROC',
               summation_method='interpolation',
               name=None,
               dtype=None,
               thresholds=None,
               multi_label=False,
               label_weights=None):
    """Creates an `AUC` instance.
    Args:
      num_thresholds: (Optional) Defaults to 200. The number of thresholds to
        use when discretizing the roc curve. Values must be > 1.
      curve: (Optional) Specifies the name of the curve to be computed, 'ROC'
        [default] or 'PR' for the Precision-Recall-curve.
      summation_method: (Optional) Specifies the Riemann summation method used
        (https://en.wikipedia.org/wiki/Riemann_sum): 'interpolation' [default],
          applies mid-point summation scheme for `ROC`. For PR-AUC, interpolates
          (true/false) positives but not the ratio that is precision (see Davis
          & Goadrich 2006 for details); 'minoring' that applies left summation
          for increasing intervals and right summation for decreasing intervals;
          'majoring' that does the opposite.
      name: (Optional) string name of the metric instance.
      dtype: (Optional) data type of the metric result.
      thresholds: (Optional) A list of floating point values to use as the
        thresholds for discretizing the curve. If set, the `num_thresholds`
        parameter is ignored. Values should be in [0, 1]. Endpoint thresholds
        equal to {-epsilon, 1+epsilon} for a small positive epsilon value will
        be automatically included with these to correctly handle predictions
        equal to exactly 0 or 1.
      multi_label: boolean indicating whether multilabel data should be
        treated as such, wherein AUC is computed separately for each label and
        then averaged across labels, or (when False) if the data should be
        flattened into a single label before AUC computation. In the latter
        case, when multilabel data is passed to AUC, each label-prediction pair
        is treated as an individual data point. Should be set to False for
        multi-class data.
      label_weights: (optional) list, array, or tensor of non-negative weights
        used to compute AUCs for multilabel data. When `multi_label` is True,
        the weights are applied to the individual label AUCs when they are
        averaged to produce the multi-label AUC. When it's False, they are used
        to weight the individual label predictions in computing the confusion
        matrix on the flattened data. Note that this is unlike class_weights in
        that class_weights weights the example depending on the value of its
        label, whereas label_weights depends only on the index of that label
        before flattening; therefore `label_weights` should not be used for
        multi-class data.
    """
    # Validate configurations.
    if isinstance(curve, metrics_utils.AUCCurve) and curve not in list(
        metrics_utils.AUCCurve):
      raise ValueError('Invalid curve: "{}". Valid options are: "{}"'.format(
          curve, list(metrics_utils.AUCCurve)))
    if isinstance(
        summation_method,
        metrics_utils.AUCSummationMethod) and summation_method not in list(
            metrics_utils.AUCSummationMethod):
      raise ValueError(
          'Invalid summation method: "{}". Valid options are: "{}"'.format(
              summation_method, list(metrics_utils.AUCSummationMethod)))

    # Update properties.
    if thresholds is not None:
      # If specified, use the supplied thresholds.
      self.num_thresholds = len(thresholds) + 2
      thresholds = sorted(thresholds)
    else:
      if num_thresholds <= 1:
        raise ValueError('`num_thresholds` must be > 1.')

      # Otherwise, linearly interpolate (num_thresholds - 2) thresholds in
      # (0, 1).
      self.num_thresholds = num_thresholds
      thresholds = [(i + 1) * 1.0 / (num_thresholds - 1)
                    for i in range(num_thresholds - 2)]

    # Add an endpoint "threshold" below zero and above one for either
    # threshold method to account for floating point imprecisions.
    self.thresholds = [0.0 - K.epsilon()] + thresholds + [1.0 + K.epsilon()]

    if isinstance(curve, metrics_utils.AUCCurve):
      self.curve = curve
    else:
      self.curve = metrics_utils.AUCCurve.from_str(curve)
    if isinstance(summation_method, metrics_utils.AUCSummationMethod):
      self.summation_method = summation_method
    else:
      self.summation_method = metrics_utils.AUCSummationMethod.from_str(
          summation_method)
    super(AUC, self).__init__(name=name, dtype=dtype)

    # Handle multilabel arguments.
    self.multi_label = multi_label
    if label_weights is not None:
      label_weights = constant_op.constant(label_weights, dtype=self.dtype)
      checks = [
          check_ops.assert_non_negative(
              label_weights,
              message='All values of `label_weights` must be non-negative.')
      ]
      self.label_weights = control_flow_ops.with_dependencies(
          checks, label_weights)

    else:
      self.label_weights = None

    self._built = False
    if self.multi_label:
      self._num_labels = None
    else:
      self._build(None)

  def _build(self, shape):
    """Initialize TP, FP, TN, and FN tensors, given the shape of the data."""
    if self.multi_label:
      if shape.ndims != 2:
        raise ValueError('`y_true` must have rank=2 when `multi_label` is '
                         'True. Found rank %s.' % shape.ndims)
      self._num_labels = shape[1]
      variable_shape = tensor_shape.TensorShape(
          [tensor_shape.Dimension(self.num_thresholds), self._num_labels])

    else:
      variable_shape = tensor_shape.TensorShape(
          [tensor_shape.Dimension(self.num_thresholds)])
    self._build_input_shape = shape
    # Create metric variables
    self.true_positives = self.add_weight(
        'true_positives',
        shape=variable_shape,
        initializer=init_ops.zeros_initializer)
    self.true_negatives = self.add_weight(
        'true_negatives',
        shape=variable_shape,
        initializer=init_ops.zeros_initializer)
    self.false_positives = self.add_weight(
        'false_positives',
        shape=variable_shape,
        initializer=init_ops.zeros_initializer)
    self.false_negatives = self.add_weight(
        'false_negatives',
        shape=variable_shape,
        initializer=init_ops.zeros_initializer)

    if self.multi_label:
      with ops.init_scope():
        # This should only be necessary for handling v1 behavior. In v2, AUC
        # should be initialized outside of any tf.functions, and therefore in
        # eager mode.
        if not context.executing_eagerly():
          K._initialize_variables(K._get_session())  # pylint: disable=protected-access

    self._built = True

  def update_state(self, y_true, y_pred, sample_weight=None):
    """Accumulates confusion matrix statistics.
    Args:
      y_true: The ground truth values.
      y_pred: The predicted values.
      sample_weight: Optional weighting of each example. Defaults to 1. Can be a
        `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
        be broadcastable to `y_true`.
    Returns:
      Update op.
    """
    deps = []
    if not self._built:
      self._build(tensor_shape.TensorShape(y_pred.shape))

    if self.multi_label or (self.label_weights is not None):
      # y_true should have shape (number of examples, number of labels).
      shapes = [
          (y_true, ('N', 'L'))
      ]
      if self.multi_label:
        # TP, TN, FP, and FN should all have shape
        # (number of thresholds, number of labels).
        shapes.extend([(self.true_positives, ('T', 'L')),
                       (self.true_negatives, ('T', 'L')),
                       (self.false_positives, ('T', 'L')),
                       (self.false_negatives, ('T', 'L'))])
      if self.label_weights is not None:
        # label_weights should be of length equal to the number of labels.
        shapes.append((self.label_weights, ('L',)))
      deps = [
          check_ops.assert_shapes(
              shapes, message='Number of labels is not consistent.')
      ]

    # Only forward label_weights to update_confusion_matrix_variables when
    # multi_label is False. Otherwise the averaging of individual label AUCs is
    # handled in AUC.result
    label_weights = None if self.multi_label else self.label_weights
    metrics_utils.update_confusion_matrix_variables(
          {
              metrics_utils.ConfusionMatrix.TRUE_POSITIVES:
                  self.true_positives,
              metrics_utils.ConfusionMatrix.TRUE_NEGATIVES:
                  self.true_negatives,
              metrics_utils.ConfusionMatrix.FALSE_POSITIVES:
                  self.false_positives,
              metrics_utils.ConfusionMatrix.FALSE_NEGATIVES:
                  self.false_negatives,
          },
          y_true,
          y_pred,
          self.thresholds,
          sample_weight=sample_weight,
          multi_label=self.multi_label,
          label_weights=label_weights)

  def interpolate_pr_auc(self):
    """Interpolation formula inspired by section 4 of Davis & Goadrich 2006.
    https://www.biostat.wisc.edu/~page/rocpr.pdf
    Note here we derive & use a closed formula not present in the paper
    as follows:
      Precision = TP / (TP + FP) = TP / P
    Modeling all of TP (true positive), FP (false positive) and their sum
    P = TP + FP (predicted positive) as varying linearly within each interval
    [A, B] between successive thresholds, we get
      Precision slope = dTP / dP
                      = (TP_B - TP_A) / (P_B - P_A)
                      = (TP - TP_A) / (P - P_A)
      Precision = (TP_A + slope * (P - P_A)) / P
    The area within the interval is (slope / total_pos_weight) times
      int_A^B{Precision.dP} = int_A^B{(TP_A + slope * (P - P_A)) * dP / P}
      int_A^B{Precision.dP} = int_A^B{slope * dP + intercept * dP / P}
    where intercept = TP_A - slope * P_A = TP_B - slope * P_B, resulting in
      int_A^B{Precision.dP} = TP_B - TP_A + intercept * log(P_B / P_A)
    Bringing back the factor (slope / total_pos_weight) we'd put aside, we get
      slope * [dTP + intercept *  log(P_B / P_A)] / total_pos_weight
    where dTP == TP_B - TP_A.
    Note that when P_A == 0 the above calculation simplifies into
      int_A^B{Precision.dTP} = int_A^B{slope * dTP} = slope * (TP_B - TP_A)
    which is really equivalent to imputing constant precision throughout the
    first bucket having >0 true positives.
    Returns:
      pr_auc: an approximation of the area under the P-R curve.
    """
    dtp = self.true_positives[:self.num_thresholds -
                              1] - self.true_positives[1:]
    p = self.true_positives + self.false_positives
    dp = p[:self.num_thresholds - 1] - p[1:]
    prec_slope = math_ops.div_no_nan(
        dtp, math_ops.maximum(dp, 0), name='prec_slope')
    intercept = self.true_positives[1:] - math_ops.multiply(prec_slope, p[1:])

    safe_p_ratio = array_ops.where(
        math_ops.logical_and(p[:self.num_thresholds - 1] > 0, p[1:] > 0),
        math_ops.div_no_nan(
            p[:self.num_thresholds - 1],
            math_ops.maximum(p[1:], 0),
            name='recall_relative_ratio'),
        array_ops.ones_like(p[1:]))

    pr_auc_increment = math_ops.div_no_nan(
        prec_slope * (dtp + intercept * math_ops.log(safe_p_ratio)),
        math_ops.maximum(self.true_positives[1:] + self.false_negatives[1:], 0),
        name='pr_auc_increment')

    if self.multi_label:
      by_label_auc = math_ops.reduce_sum(
          pr_auc_increment, name=self.name + '_by_label', axis=0)
      if self.label_weights is None:
        # Evenly weighted average of the label AUCs.
        return math_ops.reduce_mean(by_label_auc, name=self.name)
      else:
        # Weighted average of the label AUCs.
        return math_ops.div_no_nan(
            math_ops.reduce_sum(
                math_ops.multiply(by_label_auc, self.label_weights)),
            math_ops.reduce_sum(self.label_weights),
            name=self.name)
    else:
      return math_ops.reduce_sum(pr_auc_increment, name='interpolate_pr_auc')

  def result(self):
    if (self.curve == metrics_utils.AUCCurve.PR and
        self.summation_method == metrics_utils.AUCSummationMethod.INTERPOLATION
       ):
      # This use case is different and is handled separately.
      return self.interpolate_pr_auc()

    # Set `x` and `y` values for the curves based on `curve` config.
    recall = math_ops.div_no_nan(self.true_positives,
                                 self.true_positives + self.false_negatives)
    if self.curve == metrics_utils.AUCCurve.ROC:
      fp_rate = math_ops.div_no_nan(self.false_positives,
                                    self.false_positives + self.true_negatives)
      x = fp_rate
      y = recall
    else:  # curve == 'PR'.
      precision = math_ops.div_no_nan(
          self.true_positives, self.true_positives + self.false_positives)
      x = recall
      y = precision

    # Find the rectangle heights based on `summation_method`.
    if self.summation_method == metrics_utils.AUCSummationMethod.INTERPOLATION:
      # Note: the case ('PR', 'interpolation') has been handled above.
      heights = (y[:self.num_thresholds - 1] + y[1:]) / 2.
    elif self.summation_method == metrics_utils.AUCSummationMethod.MINORING:
      heights = math_ops.minimum(y[:self.num_thresholds - 1], y[1:])
    else:  # self.summation_method = metrics_utils.AUCSummationMethod.MAJORING:
      heights = math_ops.maximum(y[:self.num_thresholds - 1], y[1:])

    # Sum up the areas of all the rectangles.
    if self.multi_label:
      riemann_terms = math_ops.multiply(x[:self.num_thresholds - 1] - x[1:],
                                        heights)
      by_label_auc = math_ops.reduce_sum(
          riemann_terms, name=self.name + '_by_label', axis=0)

      if self.label_weights is None:
        # Unweighted average of the label AUCs.
        return math_ops.reduce_mean(by_label_auc, name=self.name)
      else:
        # Weighted average of the label AUCs.
        return math_ops.div_no_nan(
            math_ops.reduce_sum(
                math_ops.multiply(by_label_auc, self.label_weights)),
            math_ops.reduce_sum(self.label_weights),
            name=self.name)
    else:
      return math_ops.reduce_sum(
          math_ops.multiply(x[:self.num_thresholds - 1] - x[1:], heights),
          name=self.name)

  def reset_states(self):
    if self.multi_label:
      K.batch_set_value([(v, np.zeros((self.num_thresholds, self._num_labels)))
                         for v in self.variables])
    else:
      K.batch_set_value([
          (v, np.zeros((self.num_thresholds,))) for v in self.variables
      ])

  def get_config(self):
    if is_tensor_or_variable(self.label_weights):
      label_weights = K.eval(self.label_weights)
    else:
      label_weights = self.label_weights
    config = {
        'num_thresholds': self.num_thresholds,
        'curve': self.curve.value,
        'summation_method': self.summation_method.value,
        # We remove the endpoint thresholds as an inverse of how the thresholds
        # were initialized. This ensures that a metric initialized from this
        # config has the same thresholds.
        'thresholds': self.thresholds[1:-1],
        'multi_label': self.multi_label,
        'label_weights': label_weights
    }
    base_config = super(AUC, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [14]:
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
           AUC(name='auc', multi_label=True, label_weights=list(class_weights.values()))]

In [15]:
question = Input(shape=(1,), dtype=tf.string, name='Question')
use = hub.KerasLayer(path_to_model, trainable=True, name='USE')
#encode questions
q_emb = use(tf.squeeze(tf.cast(question, tf.string)))
#dense layer
dropout = tf.keras.layers.Dropout(0.2)(q_emb)
preds = Dense(total, activation='softmax', name='Dense')(dropout)
model = Model(inputs=question, outputs=preds)
model.compile(loss=loss_fn, optimizer=opt, run_eagerly=True, metrics=METRICS)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Question (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
tf_op_layer_Squeeze (TensorF [None]                    0         
_________________________________________________________________
USE (KerasLayer)             (None, 512)               68927232  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
Dense (Dense)                (None, 249)               127737    
Total params: 69,054,969
Trainable params: 69,054,969
Non-trainable params: 0
_________________________________________________________________


Смотрим, что без обучение выходит

In [30]:
model.evaluate(X_test, y_test, batch_size=32)

79659/79659 [==============================] - 348s 4ms/sample - loss: 5.5318 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 19994408.0000 - fn: 79659.0000 - accuracy: 0.0025 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4848


[5.531824594546119,
 0.0,
 0.0,
 19994408.0,
 79659.0,
 0.0024981482,
 0.0,
 0.0,
 0.48478246]

Учим

In [17]:
train_hist = model.fit(X_train, y_train,
                       batch_size=512,
                       epochs=20,
                       validation_data=(X_val, y_val),
                       callbacks=[early_stopping, checkpoint, reduse_lr],
                       class_weight=class_weights)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 238973 samples, validate on 79658 samples
Epoch 1/20
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
238592/238973 [============================>.] - ETA: 5s - loss: 0.8637 - accuracy: 0.0085 - auc: 0.5000
Epoch 00001: val_auc improved from -inf to 0.46601, saving model to models/USE_classif.h5
238973/238973 [==============================] - 3627s 15ms/sample - loss: 0.8638 - accuracy: 0.0085 - auc: 0.5000 - val_loss: 0.8581 - val_accuracy: 0.0149 - val_auc: 0.4660
Epoch 2/20
  7168/238973 [..............................] - ETA: 53:55 - loss: 0.8164 - accuracy: 0.0128 - auc: 0.0976